In [ ]:
import numpy as np
import pandas as pd
import fastf1
from fastf1 import utils
from collections import defaultdict, Counter
import statsmodels.api as sm
from src.utils import get_acc_df, fuel_correction, compare_car_speeds, compute_track_dominance_multi
from src.plotset import setup_plot, save_fig, plot_track_dominance

from fastf1 import plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.collections import LineCollection
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,16,'FP2')
session.load()

In [ ]:
corners = session.get_circuit_info().corners
rotation = session.get_circuit_info().rotation

In [ ]:
ref_lap = session.laps.pick_drivers('NOR').pick_fastest().get_car_data(interpolate_edges=True).add_distance().copy()
comp_lap = session.laps.pick_drivers('LEC').pick_fastest().get_car_data(interpolate_edges=True).add_distance().copy()

In [ ]:
mult = ref_lap.Distance.iloc[-1]/comp_lap.Distance.iloc[-1]

In [ ]:
ref_dist = ref_lap.Distance.to_numpy()
ref_time = ref_lap.Time.dt.total_seconds().to_numpy()
comp_dist = (comp_lap.Distance * mult).to_numpy()
comp_time = comp_lap.Time.dt.total_seconds().to_numpy()

In [ ]:
comp_time = np.interp(ref_dist, comp_dist, comp_time)

In [ ]:
delta = comp_time - ref_time

In [ ]:
dominance = pd.Series(delta).diff().rolling(window=5,center=True).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(comp_lap['Distance'],comp_lap['Speed'],color="#FF0000",linewidth=2)
ax.plot(ref_lap['Distance'],ref_lap['Speed'],color='#FF8000',linewidth=2)

v_min = ref_lap['Speed'].min()
v_max = ref_lap['Speed'].max()
ax.vlines(x=corners['Distance'], ymin=v_min-50, ymax=v_max+30,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax.text(corner['Distance'], v_max+30, txt,
            va='center_baseline', ha='center', size=15)


ax.set_xlim(ref_dist[0],ref_dist[-1])
ax.set_ylim(ref_lap['Speed'].min() - 50, ref_lap['Speed'].max() + 50)
ax.grid(visible=False)

ax2 = ax.twinx()
ax2.plot(ref_lap['Distance'],delta,ls='--')
ax2.set_ylim(-0.5,0.5)
ax2.grid(visible=False)
ax2.axhline(y=0,lw=1,ls='--',color='#444444')

In [ ]:
from scipy.signal import savgol_filter

# Convert speed to m/s
v = ref_lap.Speed * (5/18)

# Time deltas
dt = ref_lap.Time.dt.total_seconds().diff()

# Raw acceleration (m/s^2)
a = v.diff() / dt

# Convert to g (optional)
a_g = a / 9.81

# Smooth with Savitzky–Golay filter
ref_lap['LA'] = savgol_filter(a_g.fillna(0), window_length=6, polyorder=2)


# Convert speed to m/s
v = comp_lap.Speed * (5/18)

# Time deltas
dt = comp_lap.Time.dt.total_seconds().diff()

# Raw acceleration (m/s^2)
a = v.diff() / dt

# Convert to g (optional)
a_g = a / 9.81

# Smooth with Savitzky–Golay filter
comp_lap['LA'] = savgol_filter(a_g.fillna(0), window_length=6, polyorder=2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(ref_lap['Distance'],ref_lap['LA'],color='#FF8000',linewidth=2)
ax.plot(comp_lap['Distance'],comp_lap['LA'],color="#FF0000",linewidth=2)


v_min = ref_lap['LA'].min()
v_max = ref_lap['LA'].max()
ax.vlines(x=corners['Distance'], ymin=v_min-1, ymax=v_max+1,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax.text(corner['Distance'], v_max+1, txt,
            va='center_baseline', ha='center', size=15)


ax.set_xlim(ref_dist[0],ref_dist[-1])
ax.set_ylim(ref_lap['LA'].min() - 2, ref_lap['LA'].max() + 2)
ax.grid(visible=False)